# HW3


<b> Gamze Atmaca </b>, <b> Bulut Fıçıcı </b>


Since Random Forest and Gradient Descent rules takes huge space in pdf, provided them in .ipynb file. 
This pdf has the rules of Decision Tree only.

In [1]:
import sys
from pyspark import SparkConf, SparkContext, SQLContext
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import corr
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [3]:
sc = SparkContext.getOrCreate()
sqlcont = SQLContext(sc)

In [4]:
# Using schema to create df

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_btc",DoubleType(),True) \
      .add("Volume",DoubleType(),True) \

df_btc = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/btc_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_eth",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_eth = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/eth_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_xrp",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_xrp = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/xrp_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_ada",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_ada = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/ada_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_bnb",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_bnb = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/bnb_2021_hourly.csv')

In [5]:
df_btc=df_btc.drop("Open").drop("High").drop("Low").drop("Volume")
df_eth=df_eth.drop("Open").drop("High").drop("Low").drop("Volume")
df_xrp=df_xrp.drop("Open").drop("High").drop("Low").drop("Volume")
df_ada=df_ada.drop("Open").drop("High").drop("Low").drop("Volume")
df_bnb=df_bnb.drop("Open").drop("High").drop("Low").drop("Volume")

In [6]:
# Joined dataframes with respect to their Open_Time

df=df_btc.join(df_bnb,["Open_Time"])
df=df.join(df_eth,["Open_Time"])
df=df.join(df_ada,["Open_Time"])
df=df.join(df_xrp,["Open_Time"])

In [7]:
# Timestamp to datetime

df=df.withColumn("Open_Time", from_unixtime(col("Open_Time")))

In [8]:
df.show(3,vertical=True)

-RECORD 0------------------------
 Open_Time | 2021-01-01 01:00:00 
 Close_btc | 29409.99            
 Close_bnb | 37.6134             
 Close_eth | 733.37              
 Close_ada | 0.18358             
 Close_xrp | 0.22382             
-RECORD 1------------------------
 Open_Time | 2021-01-01 02:00:00 
 Close_btc | 29194.65            
 Close_bnb | 37.96               
 Close_eth | 742.27              
 Close_ada | 0.18368             
 Close_xrp | 0.22361             
-RECORD 2------------------------
 Open_Time | 2021-01-01 03:00:00 
 Close_btc | 29278.4             
 Close_bnb | 37.925              
 Close_eth | 743.1               
 Close_ada | 0.18292             
 Close_xrp | 0.2273              
only showing top 3 rows



In [9]:
# There is no NULL value in out dataset.

df.filter(df["Open_Time"].isNull()).count()+df.filter(df_btc["Close_btc"].isNull()).count()\
+df.filter(df["Close_bnb"].isNull()).count()+df.filter(df["Close_eth"].isNull()).count()\
+df.filter(df["Close_ada"].isNull()).count()+df.filter(df["Close_xrp"].isNull()).count()

0

In [10]:
# Shape of our dataset

(df.count(),len(df.columns))

(3632, 6)

In [11]:
# All values are distinct

df.distinct().count()

3632

In [12]:
# Description 

df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+-------------------+-------------------+
|summary|          Open_Time|        Close_btc|         Close_bnb|         Close_eth|          Close_ada|          Close_xrp|
+-------+-------------------+-----------------+------------------+------------------+-------------------+-------------------+
|  count|               3632|             3632|              3632|              3632|               3632|               3632|
|   mean|               null|47765.06958700446| 289.3430961178408|1988.5438436123306|  1.052209691629956| 0.7552170870044059|
| stddev|               null|9755.696878505894|199.37845900084326| 735.3322081769288|0.48754714146259637|0.46263710089047844|
|    min|2021-01-01 01:00:00|         29000.01|           35.8756|            714.29|            0.17064|            0.21743|
|    max|2021-06-01 15:00:00|         64577.26|            684.22|           4297.42|             2.4197|            1

In [13]:
# Max values of BTCUSDT

df.orderBy("Close_btc",ascending=False).show(10)

+-------------------+---------+---------+---------+---------+---------+
|          Open_Time|Close_btc|Close_bnb|Close_eth|Close_ada|Close_xrp|
+-------------------+---------+---------+---------+---------+---------+
|2021-04-14 06:00:00| 64577.26| 583.1688|  2353.56|   1.5195|  1.89768|
|2021-04-14 11:00:00| 64511.21| 574.7138|  2357.99|  1.47108|  1.83783|
|2021-04-14 07:00:00|  64288.8| 575.7693|  2369.75|  1.50496|  1.90294|
|2021-04-14 04:00:00| 64268.97| 574.2624|   2338.0|  1.53841|  1.93777|
|2021-04-14 10:00:00| 64099.99| 571.4354|  2332.74|  1.48703|  1.82924|
|2021-04-14 05:00:00| 64038.49| 579.6999|   2353.0|  1.50858|  1.89836|
|2021-04-14 08:00:00| 63928.57| 580.8758|  2327.75|  1.49377|  1.81327|
|2021-04-14 12:00:00| 63871.42| 565.0001|   2350.0|  1.42307|  1.77777|
|2021-04-14 13:00:00| 63772.67|  555.727|  2350.32|  1.41792|  1.73382|
|2021-04-14 03:00:00| 63749.19| 568.3266|  2290.25|   1.4589|  1.81586|
+-------------------+---------+---------+---------+---------+---

In [14]:
# BNB has the greatest correlation with BTC

df.select(corr("Close_btc","Close_bnb")).show()
df.select(corr("Close_btc","Close_eth")).show()
df.select(corr("Close_btc","Close_xrp")).show()
df.select(corr("Close_btc","Close_ada")).show()

+--------------------------+
|corr(Close_btc, Close_bnb)|
+--------------------------+
|        0.6851279938636956|
+--------------------------+

+--------------------------+
|corr(Close_btc, Close_eth)|
+--------------------------+
|       0.46032378356795545|
+--------------------------+

+--------------------------+
|corr(Close_btc, Close_xrp)|
+--------------------------+
|         0.516116173912515|
+--------------------------+

+--------------------------+
|corr(Close_btc, Close_ada)|
+--------------------------+
|        0.5841987920185967|
+--------------------------+



## Pipeline

In [15]:
def assemble_vectors(df, features_list, target_variable_name):
    
    stages=[]
    assembler=VectorAssembler(inputCols=features_list, outputCol="features")
    stages=[assembler]
    selectedCols=[target_variable_name,"features"]+features_list
    pipeline=Pipeline(stages=stages)
    assembleModel=pipeline.fit(df)
    df=assembleModel.transform(df).select(selectedCols)
    
    return(df)

In [16]:
x=assemble_vectors(df,["Close_bnb","Close_eth","Close_xrp","Close_ada"], "Close_btc")

In [17]:
x.show()

+---------+--------------------+---------+---------+---------+---------+
|Close_btc|            features|Close_bnb|Close_eth|Close_xrp|Close_ada|
+---------+--------------------+---------+---------+---------+---------+
| 29409.99|[37.6134,733.37,0...|  37.6134|   733.37|  0.22382|  0.18358|
| 29194.65|[37.96,742.27,0.2...|    37.96|   742.27|  0.22361|  0.18368|
|  29278.4|[37.925,743.1,0.2...|   37.925|    743.1|   0.2273|  0.18292|
| 29220.31|[37.8702,739.3,0....|  37.8702|    739.3|  0.23898|   0.1818|
| 29187.01|[37.7129,739.5,0....|  37.7129|    739.5|  0.23819|  0.18297|
| 29174.35|[37.6463,737.04,0...|  37.6463|   737.04|  0.23428|    0.181|
| 29092.83|[37.6812,734.4,0....|  37.6812|    734.4|  0.22976|  0.17955|
| 29000.01|[37.4172,725.1,0....|  37.4172|    725.1|  0.22874|  0.17762|
| 29202.21|[37.9477,728.77,0...|  37.9477|   728.77|  0.23259|  0.17904|
| 29223.82|[38.827,733.27,0....|   38.827|   733.27|  0.23692|  0.18021|
| 29313.49|[38.3904,736.11,0...|  38.3904|   736.11

## Linear Regression

In [18]:
# Linear regression model and fit 

train_df,test_df=x.randomSplit([0.7,0.3])
reg=LinearRegression(featuresCol="features", labelCol="Close_btc")
linear=reg.fit(train_df)

In [19]:
print("Intercept is:",linear.intercept)
print("\nCoefficients are:", linear.coefficients)

Intercept is: 46222.957784438855

Coefficients are: [84.24381814584642,-7.398178040990836,-15380.09655605933,3219.4486557043347]


In [20]:
test_predictions=linear.evaluate(test_df)
print("R-square value:",test_predictions.r2)

R-square value: 0.6481004021160544


In [21]:
test_predictions.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-12064.491863236359|
|-11886.962444948258|
|-11781.534865653186|
|-11794.032360291549|
|-11503.068876876277|
+-------------------+
only showing top 5 rows



## Logistic Regression

In [22]:
# Creating a new dataframe to use logistic regression. 

schema = StructType() \
      .add("Open Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close",DoubleType(),True) \
      .add("Volume",DoubleType(),True) \
      .add("Close Time",DoubleType(),True) \
      .add("Quote asset volume",DoubleType(),True) \
      .add("Number of trades",IntegerType(),True) \
      .add("Taker buy base asset volume",DoubleType(),True) \
      .add("Taker buy quote asset volume",DoubleType(),True) \
      .add("Ignore",StringType(),True) \

      
btc_df = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/btc_2021_hourly.csv')

In [23]:
# Using different columns for logistic regression.

btc_df=btc_df.drop("Open Time").drop("High").drop("Low").drop("Close Time").drop("Ignore")
btc_df.show(3, vertical=True)

-RECORD 0-------------------------------------------
 Open                         | 28995.13            
 Close                        | 29409.99            
 Volume                       | 5403.068471         
 Quote asset volume           | 1.583578168180572E8 
 Number of trades             | 103896              
 Taker buy base asset volume  | 3160.041701         
 Taker buy quote asset volume | 9.261399193555292E7 
-RECORD 1-------------------------------------------
 Open                         | 29410.0             
 Close                        | 29194.65            
 Volume                       | 2384.23156          
 Quote asset volume           | 6.98426536734203E7  
 Number of trades             | 57646               
 Taker buy base asset volume  | 1203.433506         
 Taker buy quote asset volume | 3.525274990832606E7 
-RECORD 2-------------------------------------------
 Open                         | 29195.25            
 Close                        | 29278.4       

In [24]:
# Target variable is Up_Down which takes the value of 1 if BTC rises, 0 if it falls between the opening and closing time.

btc_df=btc_df.withColumn("Up_Down", \
   when(((btc_df.Close-btc_df.Open) >= 0), lit(1)) \
     .when(((btc_df.Close-btc_df.Open) < 0), lit(0)) \
  )

In [25]:
btc_df.show(2, vertical=True)

-RECORD 0-------------------------------------------
 Open                         | 28995.13            
 Close                        | 29409.99            
 Volume                       | 5403.068471         
 Quote asset volume           | 1.583578168180572E8 
 Number of trades             | 103896              
 Taker buy base asset volume  | 3160.041701         
 Taker buy quote asset volume | 9.261399193555292E7 
 Up_Down                      | 1                   
-RECORD 1-------------------------------------------
 Open                         | 29410.0             
 Close                        | 29194.65            
 Volume                       | 2384.23156          
 Quote asset volume           | 6.98426536734203E7  
 Number of trades             | 57646               
 Taker buy base asset volume  | 1203.433506         
 Taker buy quote asset volume | 3.525274990832606E7 
 Up_Down                      | 0                   
only showing top 2 rows



In [26]:
# Using pipeline to get the features vector

model_btc=assemble_vectors(btc_df,["Volume","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume"],"Up_Down")
model_btc.show(3,vertical=True)

-RECORD 0--------------------------------------------
 Up_Down                      | 1                    
 features                     | [5403.068471,1.58... 
 Volume                       | 5403.068471          
 Quote asset volume           | 1.583578168180572E8  
 Number of trades             | 103896               
 Taker buy base asset volume  | 3160.041701          
 Taker buy quote asset volume | 9.261399193555292E7  
-RECORD 1--------------------------------------------
 Up_Down                      | 0                    
 features                     | [2384.23156,6.984... 
 Volume                       | 2384.23156           
 Quote asset volume           | 6.98426536734203E7   
 Number of trades             | 57646                
 Taker buy base asset volume  | 1203.433506          
 Taker buy quote asset volume | 3.525274990832606E7  
-RECORD 2--------------------------------------------
 Up_Down                      | 1                    
 features                   

In [27]:
trainbtc_df,testbtc_df=model_btc.randomSplit([0.7,0.3])
logreg=LogisticRegression(featuresCol="features", labelCol="Up_Down").fit(trainbtc_df)

In [28]:
train_results = logreg.evaluate(trainbtc_df).predictions

In [29]:
train_results.filter(train_results["Up_Down"]==1).filter(train_results['prediction']==1).select(['Up_Down', 'prediction', 'probability']).show(10, False)

+-------+----------+----------------------------------------+
|Up_Down|prediction|probability                             |
+-------+----------+----------------------------------------+
|1      |1.0       |[0.45978427249452236,0.5402157275054776]|
|1      |1.0       |[0.47036194790358293,0.529638052096417] |
|1      |1.0       |[0.42105595492829706,0.5789440450717029]|
|1      |1.0       |[0.4806316246407572,0.5193683753592429] |
|1      |1.0       |[0.40472284820626686,0.5952771517937332]|
|1      |1.0       |[0.43235824283374286,0.5676417571662571]|
|1      |1.0       |[0.49715641672940925,0.5028435832705908]|
|1      |1.0       |[0.49885191785884897,0.501148082141151] |
|1      |1.0       |[0.48372801576755137,0.5162719842324486]|
|1      |1.0       |[0.4332447251822255,0.5667552748177744] |
+-------+----------+----------------------------------------+
only showing top 10 rows



In [30]:
correct_preds = train_results.filter(train_results["Up_Down"]==1).filter(train_results['prediction']==1).count()

In [31]:
trainbtc_df.filter(trainbtc_df["Up_Down"]==1).count()

1281

In [32]:
#true predictions to all Up_Down==1 ratio
float(correct_preds)/(trainbtc_df.filter(trainbtc_df["Up_Down"]==1).count())

0.6518345042935206

In [33]:
#test set results
results = logreg.evaluate(testbtc_df).predictions

In [34]:
results.filter(results["Up_Down"]==1).filter(results['prediction']==1).select(['Up_Down', 'prediction', 'probability']).show(10, False)

+-------+----------+----------------------------------------+
|Up_Down|prediction|probability                             |
+-------+----------+----------------------------------------+
|1      |1.0       |[0.4582443862158398,0.5417556137841602] |
|1      |1.0       |[0.48606880079326237,0.5139311992067377]|
|1      |1.0       |[0.47725931494143103,0.522740685058569] |
|1      |1.0       |[0.49680945496720785,0.5031905450327921]|
|1      |1.0       |[0.46681780147482704,0.533182198525173] |
|1      |1.0       |[0.48047445778735753,0.5195255422126425]|
|1      |1.0       |[0.48420178152343735,0.5157982184765626]|
|1      |1.0       |[0.40131594519073815,0.5986840548092618]|
|1      |1.0       |[0.4491083205666799,0.5508916794333201] |
|1      |1.0       |[0.4545514991666857,0.5454485008333143] |
+-------+----------+----------------------------------------+
only showing top 10 rows



In [35]:
correct_preds_test = results.filter(results["Up_Down"]==1).filter(results['prediction']==1).count()
#true predictions to all Up_Down==1 ratio
float(correct_preds_test)/(testbtc_df.filter(testbtc_df["Up_Down"]==1).count())

0.6363636363636364

In [36]:
#confusion matrix
true_positives =results[(results.Up_Down==1)&(results.prediction == 1)].count()
true_negatives =results[(results.Up_Down==0)&(results.prediction == 0)].count()
false_positives =results[(results.Up_Down==0)&(results.prediction == 1)].count()
false_negatives =results[(results.Up_Down==1)&(results.prediction == 0)].count()
print(true_positives)
print(true_negatives)
print(false_positives)
print(false_negatives)
print(true_positives+true_negatives+false_positives+false_negatives)
print(results.count())

364
360
174
208
1106
1106


In [37]:
#recall score
recall = float(true_positives)/(true_positives+false_negatives)
print(recall)

0.6363636363636364


In [38]:
#precision score
precision = float(true_positives)/(true_positives+false_positives)
print(precision)

0.6765799256505576


In [39]:
#accuracy
accuracy = float((true_positives+true_negatives)/(results.count()))
print(accuracy)

0.6546112115732369


## Random Forest

In [40]:
# Creating new features and model with pipeline to use Random Forest. 

model_btc=assemble_vectors(btc_df,["Volume","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume"],"Up_Down")
model_btc.show(3,vertical=True)

-RECORD 0--------------------------------------------
 Up_Down                      | 1                    
 features                     | [5403.068471,1.58... 
 Volume                       | 5403.068471          
 Quote asset volume           | 1.583578168180572E8  
 Number of trades             | 103896               
 Taker buy base asset volume  | 3160.041701          
 Taker buy quote asset volume | 9.261399193555292E7  
-RECORD 1--------------------------------------------
 Up_Down                      | 0                    
 features                     | [2384.23156,6.984... 
 Volume                       | 2384.23156           
 Quote asset volume           | 6.98426536734203E7   
 Number of trades             | 57646                
 Taker buy base asset volume  | 1203.433506          
 Taker buy quote asset volume | 3.525274990832606E7  
-RECORD 2--------------------------------------------
 Up_Down                      | 1                    
 features                   

In [41]:
#train test split
train_rf_df,test_rf_df=model_btc.randomSplit([0.7,0.3])

In [42]:
from pyspark.ml.classification import RandomForestClassifier

rf_classifier=RandomForestClassifier(featuresCol="features", labelCol="Up_Down", numTrees=50, impurity="gini").fit(train_rf_df)
rf_predictions=rf_classifier.transform(test_rf_df)

In [43]:
rf_predictions.show(3,vertical=True)

-RECORD 0--------------------------------------------
 Up_Down                      | 0                    
 features                     | [785.471694,4.468... 
 Volume                       | 785.471694           
 Quote asset volume           | 4.468027642170553E7  
 Number of trades             | 46683                
 Taker buy base asset volume  | 394.683785           
 Taker buy quote asset volume | 2.244908604071377E7  
 rawPrediction                | [19.1374798593348... 
 probability                  | [0.38274959718669... 
 prediction                   | 1.0                  
-RECORD 1--------------------------------------------
 Up_Down                      | 0                    
 features                     | [965.546625,5.429... 
 Volume                       | 965.546625           
 Quote asset volume           | 5.429546277117824E7  
 Number of trades             | 35334                
 Taker buy base asset volume  | 475.68744            
 Taker buy quote asset volum

In [44]:
rf_predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  370|
|       1.0|  719|
+----------+-----+



In [45]:
rf_predictions.select(['probability',"Up_down",'prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |Up_down|prediction|
+----------------------------------------+-------+----------+
|[0.38274959718669754,0.6172504028133025]|0      |1.0       |
|[0.3480759134053881,0.651924086594612]  |0      |1.0       |
|[0.4263609089518555,0.5736390910481445] |0      |1.0       |
|[0.3480759134053881,0.651924086594612]  |0      |1.0       |
|[0.39077614053582155,0.6092238594641785]|0      |1.0       |
|[0.45829096107290934,0.5417090389270905]|0      |1.0       |
|[0.3675310281830835,0.6324689718169165] |0      |1.0       |
|[0.402204711964393,0.597795288035607]   |0      |1.0       |
|[0.35210245675451207,0.647897543245488] |0      |1.0       |
|[0.5247436321563085,0.47525636784369146]|0      |0.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



#Evaluate the model
rf_accuracy = MulticlassClassificationEvaluator(labelCol="Up_Down", metricName="accuracy").evaluate(rf_predictions)
print("The accuracy of RF on test data is {0: .0%}".format(rf_accuracy))

In [46]:
rf_precision = MulticlassClassificationEvaluator(labelCol="Up_Down", metricName="weightedPrecision").evaluate(rf_predictions)
print("The precision on test data is {0: .0%}".format(rf_precision))

The precision on test data is  60%


In [47]:
rf_auc = BinaryClassificationEvaluator(labelCol="Up_Down").evaluate(rf_predictions)
print("The AUC: {0: .0%}".format(rf_auc))

The AUC:  61%


In [48]:
#feature importance as a Sparse Vector

rf_classifier.featureImportances

SparseVector(5, {0: 0.2119, 1: 0.1901, 2: 0.1758, 3: 0.2119, 4: 0.2102})

## Decision Trees

In [49]:
# Creating two new dataframes (logistic_df and linear_df) to use Decision Tree. 

schema = StructType() \
      .add("Open Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close",DoubleType(),True) \
      .add("Volume",DoubleType(),True) \
      .add("Close Time",DoubleType(),True) \
      .add("Quote asset volume",DoubleType(),True) \
      .add("Number of trades",IntegerType(),True) \
      .add("Taker buy base asset volume",DoubleType(),True) \
      .add("Taker buy quote asset volume",DoubleType(),True) \
      .add("Ignore",StringType(),True) \

      
logistic_df = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/btc_2021_hourly.csv')
# Using different columns for logistic regression.

logistic_df=logistic_df.drop("Open Time").drop("High").drop("Low").drop("Close Time").drop("Ignore")

# Target variable is Up_Down which takes the value of 1 if BTC rises, 0 if it falls between the opening and closing time.

logistic_df=logistic_df.withColumn("Up_Down", \
   when(((logistic_df.Close-logistic_df.Open) >= 0), lit(1)) \
     .when(((logistic_df.Close-logistic_df.Open) < 0), lit(0)) \
  )

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_btc",DoubleType(),True) \
      .add("Volume",DoubleType(),True) \

df_btc = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/btc_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_eth",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_eth = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/eth_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_xrp",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_xrp = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/xrp_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_ada",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_ada = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/ada_2021_hourly.csv')

schema = StructType() \
      .add("Open_Time",DoubleType(),True) \
      .add("Open",DoubleType(),True) \
      .add("High",DoubleType(),True) \
      .add("Low",DoubleType(),True) \
      .add("Close_bnb",DoubleType(),True) \
      .add("Volume",DoubleType(),True) 

df_bnb = sqlcont.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load('csv/bnb_2021_hourly.csv')

df_btc=df_btc.drop("Open").drop("High").drop("Low").drop("Volume")
df_eth=df_eth.drop("Open").drop("High").drop("Low").drop("Volume")
df_xrp=df_xrp.drop("Open").drop("High").drop("Low").drop("Volume")
df_ada=df_ada.drop("Open").drop("High").drop("Low").drop("Volume")
df_bnb=df_bnb.drop("Open").drop("High").drop("Low").drop("Volume")

linear_df=df_btc.join(df_bnb,["Open_Time"])
linear_df=linear_df.join(df_eth,["Open_Time"])
linear_df=linear_df.join(df_ada,["Open_Time"])
linear_df=linear_df.join(df_xrp,["Open_Time"])
linear_df=linear_df.withColumn("Open_Time", from_unixtime(col("Open_Time")))

In [50]:
model_linear=assemble_vectors(linear_df,["Close_bnb","Close_eth","Close_ada","Close_xrp"],"Close_btc")
model_logistic=assemble_vectors(logistic_df,["Volume","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume"],"Up_Down")
model_linear.show(3,vertical=True)

-RECORD 0-------------------------
 Close_btc | 29409.99             
 features  | [37.6134,733.37,0... 
 Close_bnb | 37.6134              
 Close_eth | 733.37               
 Close_ada | 0.18358              
 Close_xrp | 0.22382              
-RECORD 1-------------------------
 Close_btc | 29194.65             
 features  | [37.96,742.27,0.1... 
 Close_bnb | 37.96                
 Close_eth | 742.27               
 Close_ada | 0.18368              
 Close_xrp | 0.22361              
-RECORD 2-------------------------
 Close_btc | 29278.4              
 features  | [37.925,743.1,0.1... 
 Close_bnb | 37.925               
 Close_eth | 743.1                
 Close_ada | 0.18292              
 Close_xrp | 0.2273               
only showing top 3 rows



In [51]:
from pyspark.ml.classification import DecisionTreeClassifier

dtreeclf= DecisionTreeClassifier(featuresCol="features", labelCol="Up_Down", impurity="gini")
class_model=dtreeclf.fit(model_logistic)
class_model.featureImportances

SparseVector(5, {0: 0.3853, 1: 0.1096, 2: 0.0646, 3: 0.3358, 4: 0.1046})

In [52]:
from pyspark.ml.regression import DecisionTreeRegressor

reg_lin=DecisionTreeRegressor(featuresCol="features",labelCol="Close_btc", impurity="variance")
reg_model=reg_lin.fit(model_linear)
reg_model.featureImportances

SparseVector(4, {0: 0.8393, 1: 0.0025, 2: 0.1191, 3: 0.039})

In [53]:
#Classifier Model Tree
class_model.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b6445e1181b0, depth=5, numNodes=41, numClasses=2, numFeatures=5\n  If (feature 1 <= 4.172340693661369E8)\n   If (feature 3 <= 3241.4672215)\n    If (feature 0 <= 5953.5265545)\n     If (feature 3 <= 2464.81589)\n      If (feature 0 <= 4697.4082005)\n       Predict: 1.0\n      Else (feature 0 > 4697.4082005)\n       Predict: 0.0\n     Else (feature 3 > 2464.81589)\n      If (feature 4 <= 8.810432546349761E7)\n       Predict: 0.0\n      Else (feature 4 > 8.810432546349761E7)\n       Predict: 1.0\n    Else (feature 0 > 5953.5265545)\n     If (feature 3 <= 2942.236581)\n      Predict: 0.0\n     Else (feature 3 > 2942.236581)\n      If (feature 1 <= 1.9858397984263003E8)\n       Predict: 1.0\n      Else (feature 1 > 1.9858397984263003E8)\n       Predict: 0.0\n   Else (feature 3 > 3241.4672215)\n    If (feature 0 <= 6464.53894)\n     If (feature 2 <= 149621.5)\n      Predict: 1.0\n     Else (feature 2 > 149621.5)\n      If (feature

## Gradient Descent

In [54]:
from pyspark.ml.classification import GBTClassifier

gdc=GBTClassifier(featuresCol="features",labelCol="Up_Down")
gdmodel=gdc.fit(train_rf_df)

print("\n")
print("Number of trees created:",gdmodel.getNumTrees)
print("\n")
print(gdmodel.featureImportances)




Number of trees created: 20


(5,[0,1,2,3,4],[0.22800087754539336,0.21214142006581044,0.17190482620185268,0.21999545670412998,0.1679574194828135])
